<a href="https://colab.research.google.com/github/Praxis-QR/BDSN/blob/main/KK_C3_Spark_MySQL_Connectivity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://4.bp.blogspot.com/-gbL5nZDkpFQ/XScFYwoTEII/AAAAAAAAAGY/CcVb_HDLwvs2Brv5T4vSsUcz7O4r2Q79ACK4BGAYYCw/s1600/kk3-header00-beta.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

## Purpose
This Colab Notebook demonstrates how to 


1.   Install Spark in Google Colab
2.   Access data lying in a Table on a remote MySQL Database

see https://towardsdatascience.com/pyspark-mysql-tutorial-fa3f7c26dc7




# Install Spark

In [1]:
!apt update -qq > /dev/null
!apt install openjdk-8-jdk-headless -qq > /dev/null

# Get latest and correct version of Spark
#
# if the current version of Spark is not used, there may be errors
# check here for current versions http://apache.osuosl.org/spark
#

!wget -q http://apache.osuosl.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip3 install -q pyspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"



E: The repository 'https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release' no longer has a Release file.


     |████████████████████████████████| 281.4 MB 39 kB/s 
     |████████████████████████████████| 198 kB 55.7 MB/s 


## Download JDBC driver <br>
Available at https://dev.mysql.com/downloads/connector/j/ <br>
We use the "Platform Independent" version

In [2]:
!wget -q https://dev.mysql.com/get/Downloads/Connector-J/mysql-connector-java-8.0.28.tar.gz

In [3]:
!ls

mysql-connector-java-8.0.28.tar.gz  spark-3.1.2-bin-hadoop3.2
sample_data			    spark-3.1.2-bin-hadoop3.2.tgz


In [4]:
!gunzip mysql-connector-java-8.0.28.tar.gz
!tar xf mysql-connector-java-8.0.28.tar

##Spark to MySQL with JDBC

In [5]:
from pyspark.sql import SparkSession

In [6]:
#/content/mysql-connector-java-8.0.28/mysql-connector-java-8.0.28.jar
#spark = SparkSession.builder.master("local[*]").config('spark.ui.port', '4050').getOrCreate()
spark = SparkSession\
                    .builder\
                    .master("local[*]")\
                    .appName("Test")\
                    .config("spark.jars", "/content/mysql-connector-java-8.0.28/mysql-connector-java-8.0.28.jar")\
                    .getOrCreate()

#MySQL Database Credentials

In [8]:
# If this Notebook is not to be shared with others
# Then place your credentials here and uncomment the lines 

#hostName = 'xxxxxxxxxx'   # uncomment this line and place your own credentials here
#userName = 'xxxxxxxxxx'      # uncomment this line and place your own credentials here
#passWord = 'xxxxxxxxxx'      # uncomment this line and place your own credentials here
#dbName =  'xxxxxxxxxx'         # uncomment this line and place your own credentials here

#print(hostName,userName,dbName)
#
# If you are using the commands in this cell, then ignore the next three cells and to Install and Test 

bq3ojarbqgmkc7prtopv-mysql.services.clever-cloud.com uxnhrnbgcu47thlz bq3ojarbqgmkc7prtopv


In [18]:
#For Shared Notebooks, the credentials are stored in a file on the author's Google Drive
from google.colab import drive
drive.mount('/content/drive')

# copy credentials file
#!ls /content/drive/'My Drive'/Praxis/WebCredentials
#!cp /content/drive/'My Drive'/Praxis/WebCredentials/sqlCredentials_020221.py credentials.py
#!cp /content/drive/'My Drive'/Praxis/WebCredentials/db4freeCredentials.py credentials.py
!cp /content/drive/'My Drive'/Praxis/WebCredentials/cleverCloud.py credentials.py

# credential file for Prithwis Mukerjee
# this file needs to be uploaded into the VM
# https://drive.google.com/drive/u/0/folders/1-EKWXDjIO_wOrNR6bR0U9IXbcFh31AMT
#
from credentials import hostName,userName,passWord,dbName

print(hostName,userName,dbName)

Mounted at /content/drive
clevercloudMongoDB.py  QuandlApiRegistrations_praxis.txt
cleverCloud.py	       QuandlApiRegistrations.txt
db4freeCredentials.py  sqlCredentials_020221.py
ngrokToken.py
bq3ojarbqgmkc7prtopv-mysql.services.clever-cloud.com uxnhrnbgcu47thlz bq3ojarbqgmkc7prtopv


In [19]:
url = 'jdbc:mysql://'+hostName+':3306/'+dbName
print(url)

jdbc:mysql://bq3ojarbqgmkc7prtopv-mysql.services.clever-cloud.com:3306/bq3ojarbqgmkc7prtopv


In [20]:
Emp_df = \
        spark.read.format("jdbc")\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("url", url) \
        .option("user", userName)\
        .option("password", passWord)\
        .option("dbtable", "Emp") \
        .load()

In [21]:
Emp_df.show()

+------+-----------+---------+----------+----------+------+----+------+
| EmpID|   LastName|FirstName|   JobDesc|  JoinDate|Salary|Comm|DeptID|
+------+-----------+---------+----------+----------+------+----+------+
|742866|    Bacchan|  Amitabh| Executive|2003-03-10| 50000| 0.1|    10|
|349870|  Mukherjee|     Rani|   Manager|2005-05-04| 25000|0.06|    40|
|865477|    Dikshit|  Madhuri|     Clerk|2002-04-04| 10000|0.02|    20|
|239456|       Khan| Shahrukh|   Manager|2004-01-03| 30000|0.07|    20|
|897889|     Sehwag| Virender|   Cus_Rep|2005-01-02| 15000|0.05|    20|
|123980|      Dhoni| Mahender|     Clerk|2004-10-09|  9000|0.02|    40|
|822134|     Dravid|    Rahul|Sr Manager|2000-06-04| 40000|0.08|    30|
|997445|     Dalmia| Jagmohan|     Clerk|2001-07-01| 12000|0.02|    30|
|989007|    Ganguly|   Sourav|   Cus_Rep|2002-01-01| 20000|0.03|    40|
|299034|    Ganesan|    Rekha|  Director|2002-10-10| 60000|0.11|    10|
|546223|Karthikeyan|  Narayan| Secretary|2005-12-04| 40000|0.09|

#SQL in MySQL from Spark

In [26]:
Select_df = \
        spark.read.format("jdbc")\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("url", url) \
        .option("user", userName)\
        .option("password", passWord)\
        .option("dbtable", "(select LastName, Salary from Emp) as emp2") \
        .load()

In [30]:
Select_df.show()

+-----------+------+
|   LastName|Salary|
+-----------+------+
|  Bacchan22| 50000|
|  Mukherjee| 25000|
|    Dikshit| 10000|
|       Khan| 30000|
|     Sehwag| 15000|
|      Dhoni|  9000|
|     Dravid| 40000|
|     Dalmia| 12000|
|    Ganguly| 20000|
|    Ganesan| 60000|
|Karthikeyan| 40000|
|      Mirza| 25000|
+-----------+------+



In [32]:
DeptSalary_df = \
        spark.read.format("jdbc")\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("url", url) \
        .option("user", userName)\
        .option("password", passWord)\
        .option("dbtable", "(select DeptID, sum(Salary) from Emp group by DeptID) as DeptSalary") \
        .load()

In [33]:
DeptSalary_df.show()

+------+-----------+
|DeptID|sum(Salary)|
+------+-----------+
|    10|     150000|
|    40|      54000|
|    20|      55000|
|    30|      77000|
+------+-----------+



#MySQL from Python Pandas

In [22]:
!apt install libmysqlclient-dev > /dev/null
!pip install mysqlclient > /dev/null
import pandas as pd
import MySQLdb

In [23]:
DBConn= MySQLdb.connect(host=hostName,port=3306,user=userName,passwd=passWord,db=dbName)
df_mysql = pd.read_sql('show tables', con=DBConn)    
print ('loaded dataframe from MySQL. records:', len(df_mysql))
DBConn.close()
df_mysql

loaded dataframe from MySQL. records: 4


,Tables_in_bq3ojarbqgmkc7prtopv
0,Dept
1,Emp
2,customers
3,itemsordered


## Pandas Functions

In [24]:
#To run any non-SELECT SQL command
def runCMD (DDL):
    DBConn= MySQLdb.connect(host=hostName,port=3306,user=userName,passwd=passWord,db=dbName)
    myCursor = DBConn.cursor()
    retcode = myCursor.execute(DDL) 
    print (retcode)
    DBConn.commit()
    DBConn.close()

#To run any SELECT SQL command
def runSELECT (CMD):
    DBConn= MySQLdb.connect(host=hostName,port=3306,user=userName,passwd=passWord,db=dbName)
    df_mysql = pd.read_sql(CMD, con=DBConn)    
    DBConn.close()
    return df_mysql

##DDL with Pandas

In [36]:
runCMD("DROP TABLE IF EXISTS Emp;")
runCMD("CREATE TABLE IF NOT EXISTS Emp ( \
  EmpID char(6)  NOT NULL, \
  LastName varchar(50) , \
  FirstName varchar(50) , \
  JobDesc varchar(50) , \
  JoinDate date NOT NULL, \
  Salary int(11) , \
  Comm float , \
  DeptID char(2) \
) ;")

0
0


In [37]:
runCMD("INSERT INTO Emp (EmpID, LastName, FirstName, JobDesc, JoinDate, Salary, Comm, DeptID) \
    VALUES \
	('742866', 'Bacchan', 'Amitabh', 'Executive', '2003-03-10', 50000, 0.1, '10'), \
	('349870', 'Mukherjee', 'Rani', 'Manager', '2005-05-04', 25000, 0.06, '40'), \
	('865477', 'Dikshit', 'Madhuri', 'Clerk', '2002-04-04', 10000, 0.02, '20'), \
	('239456', 'Khan', 'Shahrukh', 'Manager', '2004-01-03', 30000, 0.07, '20'), \
	('897889', 'Sehwag', 'Virender', 'Cus_Rep', '2005-01-02', 15000, 0.05, '20'), \
	('123980', 'Dhoni', 'Mahender', 'Clerk', '2004-10-09', 9000, 0.02, '40'), \
	('822134', 'Dravid', 'Rahul', 'Sr Manager', '2000-06-04', 40000, 0.08, '30'), \
	('997445', 'Dalmia', 'Jagmohan', 'Clerk', '2001-07-01', 12000, 0.02, '30'), \
	('989007', 'Ganguly', 'Sourav', 'Cus_Rep', '2002-01-01', 20000, 0.03, '40'), \
	('299034', 'Ganesan', 'Rekha', 'Director', '2002-10-10', 60000, 0.11, '10'), \
	('546223', 'Karthikeyan', 'Narayan', 'Secretary', '2005-12-04', 40000, 0.09, '10'), \
	('223112', 'Mirza', 'Sania', 'Cus_Rep', '2001-11-19', 25000, 0.04, '30');"
    )

12


In [ ]:
runCMD("DROP TABLE IF EXISTS Dept;")
runCMD("CREATE TABLE Dept ( \
  DeptID char(2)  NOT NULL, \
  DeptName varchar(50) , \
  ManagerID char(6) , \
  Location varchar(50) \
 );")

0
0


In [ ]:
runCMD("INSERT INTO Dept (DeptID, DeptName, ManagerID, Location) VALUES \
	('10', 'Corporate', '299034', 'Calcutta'), \
	('20', 'Sales', '239456', 'Calcutta'), \
	('30', 'Accounts', '822134', 'Calcutta'), \
	('40', 'Production', '349870', 'Bombay');")

4


In [29]:
runSELECT('Select * from Emp;')

,EmpID,LastName,FirstName,JobDesc,JoinDate,Salary,Comm,DeptID
0,742866,Bacchan22,Amitabh,Executive,2003-03-10,50000,0.10,10
1,349870,Mukherjee,Rani,Manager,2005-05-04,25000,0.06,40
2,865477,Dikshit,Madhuri,Clerk,2002-04-04,10000,0.02,20
3,239456,Khan,Shahrukh,Manager,2004-01-03,30000,0.07,20
4,897889,Sehwag,Virender,Cus_Rep,2005-01-02,15000,0.05,20
5,123980,Dhoni,Mahender,Clerk,2004-10-09,9000,0.02,40
6,822134,Dravid,Rahul,Sr Manager,2000-06-04,40000,0.08,30
7,997445,Dalmia,Jagmohan,Clerk,2001-07-01,12000,0.02,30
8,989007,Ganguly,Sourav,Cus_Rep,2002-01-01,20000,0.03,40
9,299034,Ganesan,Rekha,Director,2002-10-10,60000,0.11,10


In [ ]:
runSELECT('Select * from Dept;')

,DeptID,DeptName,ManagerID,Location
0,10,Corporate,299034,Calcutta
1,20,Sales,239456,Calcutta
2,30,Accounts,822134,Calcutta
3,40,Production,349870,Bombay


## Modify MySQL Tables

In [28]:
runCMD("update Emp set LastName = 'Bacchan22' where LastName = 'Bacchan';")

1


In [ ]:
DBConn = MySQLdb.connect(db='praxisDB')
#df_mysql = pd.read_sql('select * from emp2;', con=con_mysql)  
pd.read_sql('show tables', con=DBConn)    
#print ('loaded dataframe from MySQL. records:', len(df_mysql))
#DBConn.close()

,Tables_in_praxisDB
0,Dept
1,Emp
2,dept
3,empl


#SQL in Spark SQL

In [34]:
Emp_df.createOrReplaceTempView('Emp_T')

In [38]:
#group by gender 
spark.sql("Select * from Emp_T where DeptID = '10'").show()

+------+-----------+---------+---------+----------+------+----+------+
| EmpID|   LastName|FirstName|  JobDesc|  JoinDate|Salary|Comm|DeptID|
+------+-----------+---------+---------+----------+------+----+------+
|742866|    Bacchan|  Amitabh|Executive|2003-03-10| 50000| 0.1|    10|
|299034|    Ganesan|    Rekha| Director|2002-10-10| 60000|0.11|    10|
|546223|Karthikeyan|  Narayan|Secretary|2005-12-04| 40000|0.09|    10|
+------+-----------+---------+---------+----------+------+----+------+



In [ ]:
#group by gender 
spark.sql(\
          "SELECT \
           gender, count(gender) as count_gender, \
           count(gender)*100/sum(count(gender)) over() as percent  \
           FROM heart_T GROUP BY gender" \
           ).show()

#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself. 
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)